## A comparison between Pytorch ,TVM and Auto-TVM for timing in classification of a small model like resnet-18

In [1]:
import tvm
from tvm import relay, autotvm
import tvm.relay.testing
from tvm.autotvm.tuner import XGBTuner, GATuner, RandomTuner, GridSearchTuner
import tvm.contrib.graph_executor as runtime
import tvm.auto_scheduler as auto_scheduler
import sqlite3
import os
import random
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
import timeit
import numpy as np
from pathlib import Path
import urllib.request
import onnx

import torch
from torchvision import models
from torchvision.io import read_image
from torchvision import transforms
from torch import Tensor
from torch import nn
import torch.backends.cudnn as cudnn
import requests
import ml_dtypes

from tvm.contrib.download import download_testdata




def main():
    model = "vit"
    target = tvm.target.Target("cuda")
    if str(target).split()[0] == "cuda":
        target_device = "cuda"
    else:
        target_device = "llvm"

    batch_size = 50
    dtype = "float32"
    db_path = '/home1/public/misampson/resnet-50/git/ITE-Forth-CARV/tvm_report/automate_tvm.db'

    tuning_option = {
        "log_filename": f"{model}.log",
        "tuner": "xgb_rank_itervar",
        "n_trial": 2000,
        "early_stopping": 400,
        "measure_option": autotvm.measure_option(
            builder=autotvm.LocalBuilder(timeout=10),
            runner=autotvm.LocalRunner(number=5, repeat=1, timeout=4, min_repeat_ms=150),
        ),
    }
    transform = transforms.Compose([            
     transforms.Resize(256),                    
     transforms.CenterCrop(224),                
     transforms.ToTensor(),                     
     transforms.Normalize(                      
     mean=[0.485, 0.456, 0.406],                
     std=[0.229, 0.224, 0.225]                  
    )])
    load_random_images(batch_size)
    imgs,classes, module=run_tvm(model,get_images(),batch_size)
    # lib =tune(tuning_option, batch_size, model, target_device, db_path, target)
    if ( lib ):
        print(lib)

def tvm_relay(model,batch_size):
    target = tvm.target.Target("cuda")
    mod, params, input_shape, output_shape = get_network(model, batch_size)
    with tvm.transform.PassContext(opt_level=3):
        lib = relay.build(mod["main"], target=target, params=params)
    if ( lib ):
        print(lib)
    return lib
    
def run_module(mod):
    mod.run()
    return mod

def create_module(lib, imgs, input_name, batchsize):
    dtype = "float32"
    module = graph_executor.GraphModule(lib["default"](dev))
    images_cpu = imgs.cpu()
    images_np = np.array(images_cpu).reshape((batchsize, 3, 224, 224))
    
    # Convert NumPy array to TVM tensor
    images_tvm = tvm.nd.array(images_np.astype(dtype))
    
    module.set_input(input_name, images_tvm)
    mod = run_module(module)
    output = mod.get_output(0).asnumpy()
    prediction = np.argmax(output, axis=1)
    classes = prediction_to_class(prediction)
    return classes, module
    
def run_tvm(model,imgs,batchsize):
    device = check_device()
    imgs=imgs.to(device)
    lib, inp_name = tvm_relay(model,batchsize)
    classes ,module = create_module(tvm_lib, imgs, tvm_inp_name,batchsize) 
    return imgs, classes, module

def check_device():
    if torch.cuda.is_available():
        print("CUDA (GPU) is available.")
        device = torch.device("cuda")
    else:
        print("CUDA (GPU) is not available. Using CPU instead.")
        device = torch.device("cpu")
    return device
    
def load_random_images(batch_size):
    transform = transforms.Compose([            
     transforms.Resize(256),                    
     transforms.CenterCrop(224),                
     transforms.ToTensor(),                     
     transforms.Normalize(                      
     mean=[0.485, 0.456, 0.406],                
     std=[0.229, 0.224, 0.225]                  
    )])
    directory = "/home1/public/misampson/dataset/ILSVRC2015/Data/DET/test"
    files = os.listdir(directory)
    image_files = [f for f in files if f.endswith('.JPEG')]

    if not image_files:
        print("No image files found in the directory.")
        return None
    
    imgs = []
    chosen_image_files = []
    for _ in range(batch_size):
        random_image = random.choice(image_files)
        img_path = os.path.join(directory, random_image)
        chosen_image_files.append(img_path)  # Append the chosen image file path
        img = Image.open(img_path).convert("RGB")  # Convert to RGB format
        img_reshape = img.resize((224, 224))
        img_t = transform(img_reshape)
        imgs.append(img_t)
    
    imgs = torch.stack(imgs)
    
    with open("image_files.txt", "w") as f:
        f.write("\n".join(chosen_image_files))
    
    return imgs

def get_images():
    transform = transforms.Compose([            
     transforms.Resize(256),                    
     transforms.CenterCrop(224),                
     transforms.ToTensor(),                     
     transforms.Normalize(                      
     mean=[0.485, 0.456, 0.406],                
     std=[0.229, 0.224, 0.225]                  
    )])
    directory = "/home1/public/misampson/dataset/ILSVRC2015/Data/DET/test"
    file_path = "image_files.txt"  # Changed to the relative path of image_files.txt
    with open(file_path, "r") as f:
        image_files = f.read().splitlines()
    
    imgs = []
    for image_file in image_files:
        img = Image.open(image_file).convert("RGB")  # Load the image using the file path
        img_reshape = img.resize((224, 224))
        img_t = transform(img_reshape)
        imgs.append(img_t)
    
    imgs = torch.stack(imgs)
    return imgs
def get_network(name, batch_size):
    """Get the symbol definition and random weight of a network"""
    input_shape = (batch_size, 3, 224, 224)
    output_shape = (batch_size, 1000)

    if "resnet" in name:
        n_layer = int(name.split("-")[1])
        if n_layer == 18:
            model = models.resnet18(weights='ResNet18_Weights.IMAGENET1K_V1').eval()
        elif n_layer == 34:
            model = models.resnet34(weights='ResNet34_Weights.IMAGENET1K_V1').eval()
        elif n_layer == 50:
            model = models.resnet50(weights='ResNet50_Weights.IMAGENET1K_V1').eval()
        elif n_layer == 101:
            model = models.resnet101(weights='ResNet101_Weights.IMAGENET1K_V1').eval()
        elif n_layer == 152:
            model = models.resnet152(weights='ResNet152_Weights.IMAGENET1K_V1').eval()
        else:
            raise ValueError("Unsupported model layers: " + str(n_layer))

    elif "vgg" in name:
        n_layer = int(name.split("-")[1])
        if n_layer == 11:
            model = models.vgg11(weights='VGG11_Weights.IMAGENET1K_V1').eval()
        elif n_layer == 13:
            model = models.vgg13(weights='VGG13_Weights.IMAGENET1K_V1').eval()
        elif n_layer == 16:
            model = models.vgg16(weights='VGG16_Weights.IMAGENET1K_V1').eval()
        elif n_layer == 19:
            model = models.vgg19(weights='VGG19_Weights.IMAGENET1K_V1').eval()
        else:
            raise ValueError("Unsupported model layers: " + str(n_layer))

    elif name == "mobilenet":
        model = models.mobilenet_v2(weights='MobileNet_V2_Weights.IMAGENET1K_V1').eval()
    
    elif name == "squeezenet_v1.1":
        model = models.squeezenet1_1(weights='SqueezeNet1_1_Weights.IMAGENET1K_V1').eval()
    
    elif name == "inception_v3":
        input_shape = (batch_size, 3, 299, 299)
        model = models.inception_v3(weights='Inception_V3_Weights.IMAGENET1K_V1').eval()
    
    elif name == "vit":
        model_path = "/home1/public/misampson/resnet-50/git/ITE-Forth-CARV/tvm_report/model.onnx"
        onnx_model = onnx.load(model_path)
        input_names = [input.name for input in onnx_model.graph.input]
        print("Input names in ONNX model:", input_names)
        shape_dict = {'pixel_values': input_shape}
        mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)


        return mod, params, input_shape, output_shape
        
 
         
    else:
        raise ValueError("Unsupported network: " + name)
    shape_list = [('data', input_shape)]
    input_data = torch.randn(input_shape)
    scripted_model = torch.jit.trace(model, input_data).eval()
    mod, params = relay.frontend.from_pytorch(scripted_model, shape_list)

    return mod, params, input_shape, output_shape


def tune_tasks(
    tasks,
    measure_option,
    tuner="xgb_rank_itervar",
    n_trial=1000,
    early_stopping=400,
    log_filename="tuning.log",
    use_transfer_learning=True,
):
    tmp_log_file = log_filename + ".tmp"
    if os.path.exists(tmp_log_file):
        os.remove(tmp_log_file)

    for i, tsk in enumerate(reversed(tasks)):
        prefix = "[Task %2d/%2d] " % (i + 1, len(tasks))

        if tuner == "xgb":
            tuner_obj = XGBTuner(tsk, loss_type="reg")
        elif tuner == "xgb_knob":
            tuner_obj = XGBTuner(tsk, loss_type="reg", feature_type="knob")
        elif tuner == "xgb_itervar":
            tuner_obj = XGBTuner(tsk, loss_type="reg", feature_type="itervar")
        elif tuner == "xgb_curve":
            tuner_obj = XGBTuner(tsk, loss_type="reg", feature_type="curve")
        elif tuner == "xgb_rank":
            tuner_obj = XGBTuner(tsk, loss_type="rank")
        elif tuner == "xgb_rank_knob":
            tuner_obj = XGBTuner(tsk, loss_type="rank", feature_type="knob")
        elif tuner == "xgb_rank_itervar":
            tuner_obj = XGBTuner(tsk, loss_type="rank", feature_type="itervar")
        elif tuner == "xgb_rank_curve":
            tuner_obj = XGBTuner(tsk, loss_type="rank", feature_type="curve")
        elif tuner == "xgb_rank_binary":
            tuner_obj = XGBTuner(tsk, loss_type="rank-binary")
        elif tuner == "xgb_rank_binary_knob":
            tuner_obj = XGBTuner(tsk, loss_type="rank-binary", feature_type="knob")
        elif tuner == "xgb_rank_binary_itervar":
            tuner_obj = XGBTuner(tsk, loss_type="rank-binary", feature_type="itervar")
        elif tuner == "xgb_rank_binary_curve":
            tuner_obj = XGBTuner(tsk, loss_type="rank-binary", feature_type="curve")
        elif tuner == "ga":
            tuner_obj = GATuner(tsk, pop_size=100)
        elif tuner == "random":
            tuner_obj = RandomTuner(tsk)
        elif tuner == "gridsearch":
            tuner_obj = GridSearchTuner(tsk)
        else:
            raise ValueError("Invalid tuner: " + tuner)

        if use_transfer_learning:
            if os.path.isfile(tmp_log_file):
                tuner_obj.load_history(autotvm.record.load_from_file(tmp_log_file))

        tsk_trial = min(n_trial, len(tsk.config_space))
        tuner_obj.tune(
            n_trial=tsk_trial,
            early_stopping=early_stopping,
            measure_option=measure_option,
            callbacks=[
                autotvm.callback.progress_bar(tsk_trial, prefix=prefix),
                autotvm.callback.log_to_file(tmp_log_file),
            ],
        )

    autotvm.record.pick_best(tmp_log_file, log_filename)
    os.remove(tmp_log_file)

def tune(tuning_opt, batch_size, model, target_device, db_path, target):
    if model_exists_in_db(target_device, model, batch_size, db_path):
        print(f"Model {model} with batch size {batch_size} already exists in the database.")
        
    else:
        print("Extract tasks...")
        mod, params, input_shape, output_shape = get_network(model, batch_size)
        tasks = autotvm.task.extract_from_program(
                mod["main"], target=target, params=params, ops=None
        )
        log_file = tuning_opt["log_filename"]
        log_dir = os.path.dirname(log_file)
        if log_dir and not os.path.exists(log_dir):
            os.makedirs(log_dir)
        # Ensure the log file exists
        if not os.path.isfile(log_file):
            Path(log_file).touch()
        
        print(f"Start tuning {model} for {target_device} with batch size {batch_size}...")
        tune_tasks(tasks, **tuning_opt)
        with autotvm.apply_history_best(log_file):
            print("Compile...")
            with tvm.transform.PassContext(opt_level=3):
                lib = relay.build(mod, target=target, params=params)
        serealize_lib_to_database(target_device, model, batch_size, lib, db_path)
        return lib
        
def serealize_lib_to_database(device, network, batch_size, lib, db_path):
    lib_path = f'/home1/public/misampson/resnet-50/git/ITE-Forth-CARV/tvm_report/automated_database/{device}/{network}/{batch_size}'
    os.makedirs(lib_path, exist_ok=True)
    temp = tvm.relay.Module()
    lib = tvm.IRModule.from_expr(lib)
    file_name = "deploy.so"
    path_lib = os.path.join(lib_path, file_name)
    lib.export_library(path_lib)
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute('''
    INSERT INTO device_models (device, model, batch_size)
    VALUES (?, ?, ?)
    ''', (device, network, batch_size))

    conn.commit()
    conn.close()

def model_exists_in_db(device, network, batch_size, db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    query = '''
    SELECT COUNT(*) FROM device_models
    WHERE device = ? AND model = ? AND batch_size = ?
    '''
    cursor.execute(query, (device, network, batch_size))
    result = cursor.fetchone()[0]

    conn.close()
    return result > 0
    

if __name__ == "__main__":
    main()


CUDA (GPU) is available.
Input names in ONNX model: ['pixel_values']


One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


RuntimeError: 
#if defined(__CUDA_ARCH__) && (__CUDA_ARCH__ < 700)
#define __shfl_sync(mask, var, lane, width) \
        __shfl((var), (lane), (width))

#define __shfl_down_sync(mask, var, offset, width) \
        __shfl_down((var), (offset), (width))

#define __shfl_up_sync(mask, var, offset, width) \
        __shfl_up((var), (offset), (width))
#endif


#if (((__CUDACC_VER_MAJOR__ == 11) && (__CUDACC_VER_MINOR__ >= 4)) || \
     (__CUDACC_VER_MAJOR__ > 11))
#define TVM_ENABLE_L2_PREFETCH 1
#else
#define TVM_ENABLE_L2_PREFETCH 0
#endif

#ifdef _WIN32
  using uint = unsigned int;
  using uchar = unsigned char;
  using ushort = unsigned short;
  using int64_t = long long;
  using uint64_t = unsigned long long;
#else
  #define uint unsigned int
  #define uchar unsigned char
  #define ushort unsigned short
  #define int64_t long long
  #define uint64_t unsigned long long
#endif
extern "C" __global__ void __launch_bounds__(1024) tvmgen_default_fused_reshape_transpose_reshape_kernel(float* _

In [ ]:
import torch
from torchvision import models
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.io import read_image
from torchvision import transforms
from torch import Tensor
from torch import nn
import torch.backends.cudnn as cudnn

import os
import random
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

import timeit
import numpy as np

from transformers import ViTImageProcessor, ViTForImageClassification



## Fetch the model
The model we are using is basic but functional. Also we are not here for training classification models so we will use the pretrained resnet-18.

In [ ]:



def load_model_and_batch_size(batch_size):
    batch_size=batch_size
    model_name = "vit"
    # model_name = "resnet18"
    # model = models.resnet18(weights='ResNet18_Weights.IMAGENET1K_V1')
    #model = getattr(models, model_name)(pretrained=True)
    # model = models.vit_b_16(weights='ViT_B_16_Weights.IMAGENET1K_V1')
    # model = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)
    # model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True , autoshape=False)
    # model = models.squeezenet1_1(weights=models.SqueezeNet1_1_Weights.IMAGENET1K_V1)
    model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k", torchscript=True)
    feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

    model.eval()

    return model, batch_size

def check_device():
    if torch.cuda.is_available():
        print("CUDA (GPU) is available.")
        device = torch.device("cuda")
    else:
        print("CUDA (GPU) is not available. Using CPU instead.")
        device = torch.device("cpu")
    return device


## Load test images
Lets begin by creating some functions that will convert the image to the correct size for resnet-18

In [ ]:
 transform = transforms.Compose([            
     transforms.Resize(256),                    
     transforms.CenterCrop(224),                
     transforms.ToTensor(),                     
     transforms.Normalize(                      
     mean=[0.485, 0.456, 0.406],                
     std=[0.229, 0.224, 0.225]                  
)])

In [ ]:
def display_images_with_labels(imgs, labels, system):
    num_images = len(imgs)
    fig, axes = plt.subplots(1, num_images, figsize=(12, 4))  # Adjust figsize as needed
    
    for i, (img, label) in enumerate(zip(imgs, labels)):
        img = img.squeeze(0)  # Remove the batch dimension if it exists
        if(system=="pytorch"):
            img = img.permute(1, 2, 0)  # Change the image tensor shape from (C, H, W) to (H, W, C)
        elif(system=="tvm"):
            img = np.transpose(img, (1, 2, 0))
        else:
            print("Wrong System please select tvm or pytorch")
        # Normalize pixel values to [0, 1]
        img = img - img.min()
        img = img / img.max()

        axes[i].imshow(img)
        axes[i].axis('off')
        axes[i].set_title(label, fontsize=10, pad=5)  # Display label on top of the image
    
    plt.show()
    
def load_random_images(batch_size):
    directory = "/home1/public/misampson/dataset/ILSVRC2015/Data/DET/test"
    files = os.listdir(directory)
    image_files = [f for f in files if f.endswith('.JPEG')]

    if not image_files:
        print("No image files found in the directory.")
        return None
    
    imgs = []
    chosen_image_files = []
    for _ in range(batch_size):
        random_image = random.choice(image_files)
        img_path = os.path.join(directory, random_image)
        chosen_image_files.append(img_path)  # Append the chosen image file path
        img = Image.open(img_path).convert("RGB")  # Convert to RGB format
        img_reshape = img.resize((224, 224))
        img_t = transform(img_reshape)
        imgs.append(img_t)
    
    imgs = torch.stack(imgs)
    
    with open("image_files.txt", "w") as f:
        f.write("\n".join(chosen_image_files))
    
    return imgs

def get_images():
    directory = "/home1/public/misampson/dataset/ILSVRC2015/Data/DET/test"
    file_path = "image_files.txt"  # Changed to the relative path of image_files.txt
    with open(file_path, "r") as f:
        image_files = f.read().splitlines()
    
    imgs = []
    for image_file in image_files:
        img = Image.open(image_file).convert("RGB")  # Load the image using the file path
        img_reshape = img.resize((224, 224))
        img_t = transform(img_reshape)
        imgs.append(img_t)
    
    imgs = torch.stack(imgs)
    float_imgs=imgs.float()
    return float_imgs


## Prepare the classes
Functions that print the results of the classes

In [ ]:
def prediction_to_class(predictions):
    with open('imagenet_classes.txt') as f:
        classes = [line.strip() for line in f.readlines()]

    synsets_to_names = {}
    with open('imagenet_synsets.txt') as f:
        for line in f:
            parts = line.strip().split(' ', 1)
            synsets_to_names[parts[0]] = parts[1]

    batch_classes = []
    for prediction in predictions:
        class_name = synsets_to_names[classes[prediction]]
        batch_classes.append(class_name)

    return batch_classes


In [51]:
def timit(func, *args, **kwargs):
    timing_number = 10
    timing_repeat = 10
    
    warmup_results = timeit.repeat(lambda: func(*args, **kwargs), repeat=timing_repeat, number=timing_number)
    timing_results = timeit.repeat(lambda: func(*args, **kwargs), repeat=timing_repeat, number=timing_number)
    
    timing_summary = {
        "mean": sum(timing_results) / len(timing_results),
        "median": sorted(timing_results)[len(timing_results)//2],
        "std": np.std(timing_results),
    }
    
    print("Timing Summary:")
    print(timing_summary)
    return timing_summary


In [52]:
def run_pytorch(imgs):
    imgs=imgs.to(device)
    output = model(imgs)
    return output
    
def process_pytorch(model, batch_size):
    imgs = get_images()
    print(imgs.shape)
    labels = [] 
    out = run_pytorch(imgs)  
    #print(out.shape)
    for outputs in out:
        _, indices = torch.topk(outputs, 1)
        img_labels = prediction_to_class(indices) 
        labels.append(img_labels)
    return imgs, labels

# batch_size=50
# device = check_device()
# model, batch_size = load_model_and_batch_size(batch_size)
# model = model.to(device)
# load_random_images(batch_size)
# imgs, labels = process_pytorch(model, batch_size)
# pytime = timit(run_pytorch,imgs)
#display_images_with_labels(imgs, labels, "pytorch")

## TVM without autotuning




In [53]:
import tvm
from tvm import relay, autotvm
import tvm.relay
import tvm.relay.testing
from tvm.autotvm.tuner import XGBTuner, GATuner, RandomTuner, GridSearchTuner
import tvm.contrib.graph_executor as runtime
from tvm.contrib import graph_executor
import tvm.runtime
import pickle

import torch.utils.dlpack

from transformers import ViTModel, ViTFeatureExtractor

In [54]:

target = tvm.target.Target("cuda -arch=sm_75")
dev = tvm.cuda(0)

In [55]:
model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k", torchscript=True)
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

dummy_image = torch.randn(1, 3, 224, 224)  # Dummy image tensor
dummy_input = [dummy_image]


/home1/public/misampson/miniconda3/envs/tvm/lib/python3.11/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [56]:
# Tracing the model
traced_model = torch.jit.trace(model, dummy_input)
traced_model.eval()
for p in traced_model.parameters():
    p.requires_grad_(False)

torch.cuda.empty_cache()

In [57]:
try:
    # Move model and data to GPU
    model.cuda()
    dummy_image = dummy_image.cuda()
except RuntimeError as e:
    print(f"Error moving model to GPU: {e}")



In [58]:
shape_list = [(i.debugName().split('.')[0], i.type().sizes()) for i in list(traced_model.graph.inputs())[1:]]
mod_vit, params_vit = tvm.relay.frontend.from_pytorch(traced_model, shape_list, default_dtype="float32")

# target = tvm.target.cuda
target = tvm.target.Target("cuda -arch=sm_75")
ctx = tvm.cuda(0)
dummy_image_np = dummy_image.cpu().numpy()
dummy_image_tvm = tvm.nd.array(dummy_image_np, ctx)
# tvm.relay.backend.compile_engine.get().clear()
tvm.relay.backend.te_compiler.get().clear()
with tvm.transform.PassContext(opt_level=3):
    graph, lib, params = tvm.relay.build(mod_vit, target=target, params=params_vit)

module = tvm.contrib.graph_runtime.create(graph, lib, ctx)

module.set_input("input0", dummy_image_tvm)
module.set_input(**params)
module.run()
output = module.get_output(0)


/home1/public/misampson/miniconda3/envs/tvm/lib/python3.11/site-packages/tvm-0.17.dev2+ga64d1f1cc-py3.11-linux-x86_64.egg/tvm/target/target.py:375: UserWarning: target_host parameter is going to be deprecated. Please pass in tvm.target.Target(target, host=target_host) instead.
  warnings.warn(


RuntimeError: 
#if defined(__CUDA_ARCH__) && (__CUDA_ARCH__ < 700)
#define __shfl_sync(mask, var, lane, width) \
        __shfl((var), (lane), (width))

#define __shfl_down_sync(mask, var, offset, width) \
        __shfl_down((var), (offset), (width))

#define __shfl_up_sync(mask, var, offset, width) \
        __shfl_up((var), (offset), (width))
#endif


#if (((__CUDACC_VER_MAJOR__ == 11) && (__CUDACC_VER_MINOR__ >= 4)) || \
     (__CUDACC_VER_MAJOR__ > 11))
#define TVM_ENABLE_L2_PREFETCH 1
#else
#define TVM_ENABLE_L2_PREFETCH 0
#endif

#ifdef _WIN32
  using uint = unsigned int;
  using uchar = unsigned char;
  using ushort = unsigned short;
  using int64_t = long long;
  using uint64_t = unsigned long long;
#else
  #define uint unsigned int
  #define uchar unsigned char
  #define ushort unsigned short
  #define int64_t long long
  #define uint64_t unsigned long long
#endif
extern "C" __global__ void __launch_bounds__(1024) tvmgen_default_fused_nn_softmax_kernel(float* __restrict__ T_s

In [ ]:
res_pt = model(dummy_image)
(numpy.abs((res_pt[0].cpu().numpy() - output.asnumpy())).max())


In [ ]:
tvm_wo_autotune_time=timit(run_module,module)

In [ ]:
import threading
import time
import numpy as np
import matplotlib.pyplot as plt

def execute_and_plot_tvm_timit():
    global tvm_lib, tvm_inp_name
    batch_sizes = [1, 10, 100, 200, 256]
    timing_results = []

    for batch_size in batch_sizes:
        #we want to enter relay for each batch
        tvm_lib = None
        tvm_inp_name = None
        load_random_images(batch_size)
        model, batch_size = load_model_and_batch_size(batch_size)
        imgs,classes, module=run_tvm(get_images(),batch_size)
        tvm_wo_autotune_time=timit(run_module,module)
        mean_time = tvm_wo_autotune_time["mean"]
        save_mean_time(batch_size, mean_time)

    timing_results = get_timing_results(batch_sizes)  # Retrieve timing results from saved files
    plot_timing_results(timing_results)


def save_mean_time(batch_size, mean_time):
    with open(f'tvm_timing_batch_{batch_size}.txt', 'w') as f:
        f.write(str(mean_time))

def get_timing_results(batch_sizes):
    timing_results = []
    for batch_size in batch_sizes:
        with open(f'tvm_timing_batch_{batch_size}.txt', 'r') as f:
            mean_time = float(f.read())
        timing_results.append((batch_size, mean_time))
    return timing_results

def plot_timing_results(timing_results):
    # Remove None values from timing_results
    timing_results = [result for result in timing_results if result is not None]

    if not timing_results:
        print("No timing results to plot.")
        return

    timing_results.sort(key=lambda x: x[1])  # Sort by mean time
    batch_sizes = [result[0] for result in timing_results]  # Extract batch sizes
    timing_means = [result[1] for result in timing_results]  # Extract timing results

    plt.figure(figsize=(10, 6))

    # Generate equally spaced y-axis ticks
    y_ticks = np.arange(len(batch_sizes))

    # Plot horizontal bars for mean timing results
    colors = plt.cm.viridis(np.linspace(0, 1, len(batch_sizes)))  # Generate different colors
    for i, (mean, size) in enumerate(zip(timing_means, batch_sizes)):
        plt.barh(y_ticks[i], mean, color=colors[i], label=f'Batch Size {size}')

    # Set y-axis ticks and labels
    plt.yticks(y_ticks, batch_sizes)

    plt.title('PyTorch Mean Execution Time vs Batch Size')
    plt.xlabel('Mean Execution Time (seconds)')
    plt.ylabel('Batch Size')
    plt.legend()
    plt.grid(axis='x', linestyle='--', alpha=0.7)  # Remove background grid lines
    plt.gca().invert_yaxis()  # Invert y-axis to have the smallest batch size at the top
    plt.tight_layout()
    plt.show()

# Usage
#execute_and_plot_tvm_timit()


## Begin TVM steps




In [ ]:
import tvm.relay.testing
from tvm.autotvm.tuner import XGBTuner, GATuner, RandomTuner, GridSearchTuner
import tvm.contrib.graph_executor as runtime
import tvm.auto_scheduler as auto_scheduler
from tvm.autotvm.tuner import XGBTuner

## Define Network
First we need to define the network in relay frontend API.
We can load some pre-defined network from :code:`tvm.relay.testing`.
We can also load models from MXNet, ONNX and TensorFlow.



In [ ]:
def get_network(name, batch_size):
    """Get the symbol definition and random weight of a network"""
    input_shape = (batch_size, 3, 224, 224)
    output_shape = (batch_size, 1000)

    if "resnet" in name:
        n_layer = int(name.split("-")[1])
        mod, params = relay.testing.resnet.get_workload(
            num_layers=n_layer, batch_size=batch_size, dtype=dtype
        )
    elif "vgg" in name:
        n_layer = int(name.split("-")[1])
        mod, params = relay.testing.vgg.get_workload(
            num_layers=n_layer, batch_size=batch_size, dtype=dtype
        )
    elif name == "mobilenet":
        mod, params = relay.testing.mobilenet.get_workload(batch_size=batch_size, dtype=dtype)
    elif name == "squeezenet_v1.1":
        mod, params = relay.testing.squeezenet.get_workload(
            batch_size=batch_size, version="1.1", dtype=dtype
        )
    elif name == "inception_v3":
        input_shape = (batch_size, 3, 299, 299)
        mod, params = relay.testing.inception_v3.get_workload(batch_size=batch_size, dtype=dtype)
    elif name == "mxnet":
        # an example for mxnet model
        from mxnet.gluon.model_zoo.vision import get_model

        block = get_model("resnet18_v1", pretrained=True)
        mod, params = relay.frontend.from_mxnet(block, shape={"data": input_shape}, dtype=dtype)
        net = mod["main"]
        net = relay.Function(
            net.params, relay.nn.softmax(net.body), None, net.type_params, net.attrs
        )
        mod = tvm.IRModule.from_expr(net)
    elif name == "vit":
        model_path = "/home1/public/misampson/resnet-50/git/ITE-Forth-CARV/tvm_report/model.onnx"
        onnx_model = onnx.load(model_path)
        input_names = [input.name for input in onnx_model.graph.input]
        print("Input names in ONNX model:", input_names)
        shape_dict = {'pixel_values': input_shape}
        mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)
        return mod, params, input_shape, output_shape
        
    else:
        raise ValueError("Unsupported network: " + name)

    return mod, params, input_shape, output_shape

## Set Tuning Options
Before tuning, we apply some configurations.



In [ ]:
#### DEVICE CONFIG ####
target = tvm.target.cuda()

#### TUNING OPTION ####
network = "squeezenet_v1.1"
log_file = "%s.log" % network
dtype = "float32"

tuning_option = {
    "log_filename": log_file,
    "tuner": "xgb",
    "n_trial": 2000,
    "early_stopping": 2,
    "measure_option": autotvm.measure_option(
        builder=autotvm.LocalBuilder(timeout=10),
        runner=autotvm.LocalRunner(number=4, repeat=1, timeout=4, min_repeat_ms=150),
    ),
}

## Begin Tuning
Now we can extract tuning tasks from the network and begin tuning.
Here, we provide a simple utility function to tune a list of tasks.
This function is just an initial implementation which tunes them in sequential order.
We will introduce a more sophisticated tuning scheduler in the future.



In [ ]:
# You can skip the implementation of this function for this tutorial.
def tune_tasks(
    tasks,
    measure_option,
    tuner="xgb",
    n_trial=1000,
    early_stopping=2,
    log_filename="tuning.log",
    use_transfer_learning=True,
):
    # create tmp log file
    tmp_log_file = log_filename + ".tmp"
    if os.path.exists(tmp_log_file):
        os.remove(tmp_log_file)

    for i, tsk in enumerate(reversed(tasks)):
        prefix = "[Task %2d/%2d] " % (i + 1, len(tasks))

        # create tuner
        if tuner == "xgb":
            tuner_obj = XGBTuner(tsk, loss_type="reg")
        elif tuner == "xgb_knob":
            tuner_obj = XGBTuner(tsk, loss_type="reg", feature_type="knob")
        elif tuner == "xgb_itervar":
            tuner_obj = XGBTuner(tsk, loss_type="reg", feature_type="itervar")
        elif tuner == "xgb_curve":
            tuner_obj = XGBTuner(tsk, loss_type="reg", feature_type="curve")
        elif tuner == "xgb_rank":
            tuner_obj = XGBTuner(tsk, loss_type="rank")
        elif tuner == "xgb_rank_knob":
            tuner_obj = XGBTuner(tsk, loss_type="rank", feature_type="knob")
        elif tuner == "xgb_rank_itervar":
            tuner_obj = XGBTuner(tsk, loss_type="rank", feature_type="itervar")
        elif tuner == "xgb_rank_curve":
            tuner_obj = XGBTuner(tsk, loss_type="rank", feature_type="curve")
        elif tuner == "xgb_rank_binary":
            tuner_obj = XGBTuner(tsk, loss_type="rank-binary")
        elif tuner == "xgb_rank_binary_knob":
            tuner_obj = XGBTuner(tsk, loss_type="rank-binary", feature_type="knob")
        elif tuner == "xgb_rank_binary_itervar":
            tuner_obj = XGBTuner(tsk, loss_type="rank-binary", feature_type="itervar")
        elif tuner == "xgb_rank_binary_curve":
            tuner_obj = XGBTuner(tsk, loss_type="rank-binary", feature_type="curve")
        elif tuner == "ga":
            tuner_obj = GATuner(tsk, pop_size=100)
        elif tuner == "random":
            tuner_obj = RandomTuner(tsk)
        elif tuner == "gridsearch":
            tuner_obj = GridSearchTuner(tsk)
        else:
            raise ValueError("Invalid tuner: " + tuner)

        if use_transfer_learning:
            if os.path.isfile(tmp_log_file):
                tuner_obj.load_history(autotvm.record.load_from_file(tmp_log_file))

        # do tuning
        tsk_trial = min(n_trial, len(tsk.config_space))
        tuner_obj.tune(
            n_trial=tsk_trial,
            early_stopping=early_stopping,
            measure_option=measure_option,
            callbacks=[
                autotvm.callback.progress_bar(tsk_trial, prefix=prefix),
                autotvm.callback.log_to_file(tmp_log_file),
            ],
        )

    # pick best records to a cache file
    autotvm.record.pick_best(tmp_log_file, log_filename)
    os.remove(tmp_log_file)

In [ ]:
def tune_and_evaluate(tuning_opt,batch_size):
    # extract workloads from relay program
    print("Extract tasks...")
    mod, params, input_shape, out_shape = get_network(network, batch_size)
    tasks = autotvm.task.extract_from_program(
        mod["main"], target=target, params=params, ops=(relay.op.get("nn.conv2d"),)
    )

    # run tuning tasks
    print("Tuning...")
    tune_tasks(tasks, **tuning_opt)

    # compile kernels with history best records
    with autotvm.apply_history_best(log_file):
        print("Compile...")
        with tvm.transform.PassContext(opt_level=3):
            lib = relay.build_module.build(mod, target=target, params=params)

        # load parameters
        dev = tvm.device(str(target), 0)
        module = runtime.GraphModule(lib["default"](dev))
        data_tvm = tvm.nd.array((np.random.uniform(size=input_shape)).astype(dtype))
        module.set_input("data", data_tvm)

        # evaluate
        print("Evaluate inference time cost...")
        print(module.benchmark(dev, number=1, repeat=600))
        
    return module
# We do not run the tuning in our webpage server since it takes too long.
# Uncomment the following line to run it by yourself.
module =tune_and_evaluate(tuning_option,batch_size)


In [ ]:
def tvm_relay_with_file(network, batch_size):
    tuning_logs_dir = "/home1/public/misampson/resnet-50/git/ITE-Forth-CARV/tuning-logs"
    log_dir = f"log{batch_size}"
    logfile_path = os.path.join(tuning_logs_dir, "tune_nn/longer-tune", log_dir, "resnet-18.log")
    
    mod, params, input_shape, out_shape = get_network(network, batch_size)
    
    with autotvm.apply_history_best(logfile_path):
        with tvm.transform.PassContext(opt_level=3):
            lib = relay.build(mod, target=target, params=params)
    
    # Load the compiled module onto the device
    dev = tvm.device(str(target), 0)
    module = runtime.GraphModule(lib["default"](dev))
    return module

def save_mean_time(batch_size, mean_time):
    with open(f'autotune_all_timing_batch_{batch_size}.txt', 'w') as f:
        f.write(str(mean_time))

def get_timing_results(batch_sizes):
    timing_results = []
    for batch_size in batch_sizes:
        with open(f'autotune_all_timing_batch_{batch_size}.txt', 'r') as f:
            mean_time = float(f.read())
        timing_results.append((batch_size, mean_time))
    return timing_results
    
batch_size=256
# module = tvm_relay_with_file(network, batch_size)

In [ ]:

 tvm_wo_autotune_time=timit(run_module,module)
# mean_time = tvm_wo_autotune_time["mean"]
# save_mean_time(batch_size, mean_time)

In [ ]:
import threading
import time
import numpy as np
import matplotlib.pyplot as plt

def execute_and_plot_autotune_timit():
    global tvm_lib, tvm_inp_name
    batch_sizes = [1, 10, 100, 200, 256]
    timing_results = []

    for batch_size in batch_sizes:
        module = tvm_relay_with_file(network, batch_size)
        tvm_wo_autotune_time=timit(run_module,module)
        mean_time = tvm_wo_autotune_time["mean"]
        save_mean_time(batch_size, mean_time)

    timing_results = get_timing_results(batch_sizes)  # Retrieve timing results from saved files
    plot_timing_results(timing_results)


def save_mean_time(batch_size, mean_time):
    with open(f'autotune_timing_batch_{batch_size}.txt', 'w') as f:
        f.write(str(mean_time))

def get_timing_results(batch_sizes):
    timing_results = []
    for batch_size in batch_sizes:
        with open(f'autotune_timing_batch_{batch_size}.txt', 'r') as f:
            mean_time = float(f.read())
        timing_results.append((batch_size, mean_time))
    return timing_results

def plot_timing_results(timing_results):
    # Remove None values from timing_results
    timing_results = [result for result in timing_results if result is not None]

    if not timing_results:
        print("No timing results to plot.")
        return

    timing_results.sort(key=lambda x: x[1])  # Sort by mean time
    batch_sizes = [result[0] for result in timing_results]  # Extract batch sizes
    timing_means = [result[1] for result in timing_results]  # Extract timing results

    plt.figure(figsize=(10, 6))

    # Generate equally spaced y-axis ticks
    y_ticks = np.arange(len(batch_sizes))

    # Plot horizontal bars for mean timing results
    colors = plt.cm.viridis(np.linspace(0, 1, len(batch_sizes)))  # Generate different colors
    for i, (mean, size) in enumerate(zip(timing_means, batch_sizes)):
        plt.barh(y_ticks[i], mean, color=colors[i], label=f'Batch Size {size}')

    # Set y-axis ticks and labels
    plt.yticks(y_ticks, batch_sizes)

    plt.title('PyTorch Mean Execution Time vs Batch Size')
    plt.xlabel('Mean Execution Time (seconds)')
    plt.ylabel('Batch Size')
    plt.legend()
    plt.grid(axis='x', linestyle='--', alpha=0.7)  # Remove background grid lines
    plt.gca().invert_yaxis()  # Invert y-axis to have the smallest batch size at the top
    plt.tight_layout()
    plt.show()

# Usage
#execute_and_plot_autotune_timit()


In [ ]:
# tvm_autotune_time=timit(run_module,module)

In [ ]:
def plot_timestamps(time1, time2, time3):
    # Parse the timestamps to extract mean, median, and standard deviation values
    def parse_timestamp(timestamp):
        if timestamp is None:
            return None, None, None
        return timestamp["mean"], timestamp["median"], timestamp["std"]

    t1_mean, t1_median, t1_std = parse_timestamp(time1)
    t2_mean, t2_median, t2_std = parse_timestamp(time2)
    t3_mean, t3_median, t3_std = parse_timestamp(time3)

    # Plotting
    labels = ['Mean', 'Median', 'Std']
    t1_values = [t1_mean, t1_median, t1_std]
    t2_values = [t2_mean, t2_median, t2_std]
    t3_values = [t3_mean, t3_median, t3_std]

    x = range(len(labels))
    width = 0.2

    fig, ax = plt.subplots()
    bars = []
    
    if t1_mean is not None:
        bars.append(ax.bar(x, t1_values, width, label='Pytorch'))
    if t2_mean is not None:
        bars.append(ax.bar([i + width for i in x], t2_values, width, label='TVM without tuning'))
    if t3_mean is not None:
        bars.append(ax.bar([i + width*2 for i in x], t3_values, width, label='TVM autotune'))

    ax.set_xlabel('Metrics')
    ax.set_ylabel('Time')
    ax.set_title('Classification Timing Comparison')
    ax.set_xticks([i + width for i in x])
    ax.set_xticklabels(labels)
    
    # Add legend only if there are bars plotted
    if bars:
        ax.legend()

    plt.show()

In [ ]:
# plot_timestamps(pytorch_time, tvm_wo_autotune_time, tvm_autotune_time)

In [ ]:

def mean_timestamps(time1, time2, time3):
    # Parse the timestamps to extract mean values
    def parse_timestamp(timestamp):
        if timestamp is None:
            return None
        return timestamp["mean"]

    t1_mean = parse_timestamp(time1)
    t2_mean = parse_timestamp(time2)
    t3_mean = parse_timestamp(time3)

    # Plotting
    labels = ['Pytorch', 'TVM without tuning', 'TVM autotune']
    means = [t1_mean, t2_mean, t3_mean]

    x = range(len(labels))
    width = 0.5

    fig, ax = plt.subplots()
    bars = []

    if t1_mean is not None:
        bars.append(ax.bar(x[0], t1_mean, width, label='Pytorch'))
    if t2_mean is not None:
        bars.append(ax.bar(x[1], t2_mean, width, label='TVM without tuning'))
    if t3_mean is not None:
        bars.append(ax.bar(x[2], t3_mean, width, label='TVM autotune'))

    ax.set_xlabel('Frameworks')
    ax.set_ylabel('Mean Time')
    ax.set_title('Mean Classification Time Comparison')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    # Add the mean value on top of each bar
    for bar in bars:
        for b in bar:
            height = b.get_height()
            ax.annotate(f'{height:.4f}',
                        xy=(b.get_x() + b.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom')

    plt.show()

In [ ]:
# mean_timestamps(pytorch_time, tvm_wo_autotune_time, tvm_autotune_time)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def get_timing_results(file_prefixes, batch_sizes):
    timing_results_list = []
    for file_prefix in file_prefixes:
        timing_results = []
        for batch_size in batch_sizes:
            file_path = f'{file_prefix}{batch_size}.txt'
            try:
                with open(file_path, 'r') as f:
                    mean_time = float(f.read())
                timing_results.append((batch_size, mean_time))
            except FileNotFoundError:
                pass  # Skip if file not found for the current batch size
        timing_results_list.append(timing_results)
    return timing_results_list

def plot_timing_results(timing_results_list, labels, model, dataset):
    if not timing_results_list or not labels:
        print("No timing results or labels provided.")
        return
    
    plt.figure(figsize=(12, 8))  # Adjust figsize if needed

    # Generate equally spaced x-axis ticks
    x_ticks = np.arange(len(timing_results_list[0]))
    
    # Define colors for PyTorch, TVM, and autotune
    color_map = {'PyTorch': 'orange', 'TVM': 'lightblue', 'Autotune': 'darkblue'}
    
    # Plot vertical bars for mean timing results
    for i, (timing_results, label) in enumerate(zip(timing_results_list, labels)):
        timing_means = [result[1] for result in timing_results]
        batch_sizes = [result[0] for result in timing_results]
        
        for j, (mean, size) in enumerate(zip(timing_means, batch_sizes)):
            plt.bar(x_ticks[j] + i * 0.2, mean, color=color_map.get(label, 'black'), width=0.2)
            #plt.text(x_ticks[j] + i * 0.2, mean, f'{mean:.3f}s', ha='center', va='bottom', color='black', fontsize=8)

    # Set x-axis ticks and labels
    if 256 in batch_sizes:
        batch_sizes = [size for size in batch_sizes if size != 256] + [256]  # Move batch size 256 to the end
    plt.xticks(x_ticks + 0.2 * len(timing_results_list) / 2, batch_sizes, fontsize=12)

    plt.title('Execution on Nvidia GPU 2080ti', fontsize=36)
    plt.xlabel('Batch Size', fontsize=24)
    plt.ylabel('Mean Execution Time (seconds)', fontsize=24)
    
    # Define custom legend labels and handles with corresponding colors
    custom_handles = [plt.Rectangle((0,0),1,1, color=color_map[label]) for label in labels]
    custom_labels = labels
    
    # Display legend with custom labels and handles
    plt.legend(custom_handles, custom_labels, loc='upper left', fontsize=12)
    
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.show()

# Example usage
file_prefixes = ['pytorch_timing_batch_', 'tvm_timing_batch_', 'autotune_timing_batch_']
labels = ['PyTorch', 'TVM', 'Autotune']
batch_sizes = [1, 10, 100, 200, 256] 
timing_results_list = get_timing_results(file_prefixes, batch_sizes)
plot_timing_results(timing_results_list, labels, model='ResNet-18', dataset='ImageNet')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def get_timing_results(file_prefixes, batch_size):
    timing_results_list = []
    for file_prefix in file_prefixes:
        file_path = f'{file_prefix}{batch_size}.txt'
        try:
            with open(file_path, 'r') as f:
                mean_time = float(f.read())
            timing_results_list.append((batch_size, mean_time))
        except FileNotFoundError:
            timing_results_list.append((batch_size, None))  # Use None for missing values
    return timing_results_list

def plot_timing_results(timing_results_list, labels, model, dataset):
    if not timing_results_list or not labels:
        print("No timing results or labels provided.")
        return
    
    plt.figure(figsize=(12, 8))  # Adjust figsize if needed

    # Define colors for PyTorch, TVM, Autotune, and Autotune_all
    color_map = {'PyTorch': 'orange', 'TVM': 'lightblue', 'Autotune': 'darkblue', 'Autotune_all': 'purple'}
    
    # Plot vertical bars for mean timing results
    batch_size, _ = timing_results_list[0]  # All results should have the same batch size
    x_ticks = np.arange(1)  # Only one batch size

    # Set bar width and spacing
    bar_width = 0.05  # Thinner bars
    spacing = 0.02  # Less spacing

    for i, ((_, mean), label) in enumerate(zip(timing_results_list, labels)):
        if mean is not None:  # Only plot if mean is available
            plt.bar(x_ticks + i * (bar_width + spacing), mean, color=color_map.get(label, 'black'), width=bar_width)
            plt.text(x_ticks + i * (bar_width + spacing), mean, f'{mean:.3f}s', ha='center', va='bottom', color='black', fontsize=8)

    # Set x-axis ticks and labels
    plt.xticks([0.1], [batch_size], fontsize=12)  # Adjust position to align with thinner bars

    plt.title('Execution on Nvidia GPU 2080ti', fontsize=36)
    plt.xlabel('Batch Size', fontsize=24)
    plt.ylabel('Mean Execution Time (seconds)', fontsize=24)
    
    # Define custom legend labels and handles with corresponding colors
    custom_handles = [plt.Rectangle((0,0),1,1, color=color_map[label]) for label in labels]
    custom_labels = ['PyTorch', 'TVM', 'Autotune_conv2d', 'Autotune_all']  # Updated legend label
    
    # Display legend with custom labels and handles
    plt.legend(custom_handles, custom_labels, loc='upper left', fontsize=12)
    
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.show()

# Example usage
file_prefixes = ['pytorch_timing_batch_', 'tvm_timing_batch_', 'autotune_timing_batch_', 'autotune_all_timing_batch_']
labels = ['PyTorch', 'TVM', 'Autotune', 'Autotune_all']
batch_size = 256 
timing_results_list = get_timing_results(file_prefixes, batch_size)
plot_timing_results(timing_results_list, labels, model='ResNet-18', dataset='ImageNet')


In [ ]:
def tvm_relay_with_file(network, batch_size, logfile_path):
    mod, params, input_shape, out_shape = get_network(network, batch_size)
    if logfile_path:
        with autotvm.apply_history_best(logfile_path):
            with tvm.transform.PassContext(opt_level=3):
                lib = relay.build(mod, target=target, params=params)
    else:
        with tvm.transform.PassContext(opt_level=3):
                lib = relay.build(mod, target=target, params=params)
        
    # Load the compiled module onto the device
    dev = tvm.device(str(target), 0)
    module = runtime.GraphModule(lib["default"](dev))
    return module


batch_size=50
# Run TVM relay with two different log files
logfile_path_conv2d = "/home1/public/misampson/resnet-50/git/ITE-Forth-CARV/tuning-logs/longer-tune/log256/conv2d/vit_conv2d.log"
logfile_path_all = "/home1/public/misampson/resnet-50/git/ITE-Forth-CARV/tuning-logs/longer-tune/log256/all/vit.log"

# Generate modules
module_base = tvm_relay_with_file('vit', batch_size, None)
module_conv2d = tvm_relay_with_file('vit', batch_size, logfile_path_conv2d)
module_all = tvm_relay_with_file('vit', batch_size, logfile_path_all)

# Dummy mean times: Replace these with actual measured times
mean_pytorch = pytime['mean']
mean_time_base= timit(run_module, module_base)['mean']
mean_time_conv2d = timit(run_module, module_conv2d)['mean']
mean_time_all = timit(run_module, module_all)['mean']

# Save the mean times to variables instead of files
timing_results_list = [
    (batch_size, mean_pytorch),
    (batch_size, mean_time_base),
    (batch_size, mean_time_conv2d),
    (batch_size, mean_time_all)
]


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_timing_results(timing_results_list, labels, model, dataset):
    if not timing_results_list or not labels:
        print("No timing results or labels provided.")
        return

    plt.figure(figsize=(12, 8))  # Adjust figsize if needed

    # Define colors for PyTorch, TVM, Autotune_conv2d, and Autotune_all
    color_map = {
        'PyTorch': 'orange', 
        'TVM': 'lightblue', 
        'Autotune_conv2d': 'darkblue', 
        'Autotune_all': 'purple'
    }
    
    # Ensure PyTorch is the first label and its timing results are first in the list
    if 'PyTorch' not in labels:
        print("PyTorch timing results not provided.")
        return

    if labels[0] != 'PyTorch':
        pytorch_index = labels.index('PyTorch')
        labels.insert(0, labels.pop(pytorch_index))
        timing_results_list.insert(0, timing_results_list.pop(pytorch_index))

    # Plot vertical bars for mean timing results
    batch_size, _ = timing_results_list[0]  # All results should have the same batch size
    x_ticks = np.arange(1)  

    # Set bar width and spacing
    bar_width = 0.1  # Thinner bars
    spacing = 0.2  # Less spacing

    for i, ((_, mean), label) in enumerate(zip(timing_results_list, labels)):
        if mean is not None:  # Only plot if mean is available
            plt.bar(x_ticks + i * (bar_width + spacing), mean, color=color_map.get(label, 'black'), width=bar_width)
            plt.text(x_ticks + i * (bar_width + spacing), mean, f'{mean:.3f}s', ha='center', va='bottom', color='black', fontsize=8)

    # Set x-axis ticks and labels
    plt.xticks([0.1], [50], fontsize=12)  # Adjust position to align with thinner bars

    plt.title(f'Execution on Nvidia GPU 2080ti\nModel: {model}, Dataset: {dataset}', fontsize=24)
    plt.xlabel('Batch Size', fontsize=24)
    plt.ylabel('Mean Execution Time (seconds)', fontsize=24)
    
    # Define custom legend labels and handles with corresponding colors
    custom_handles = [plt.Rectangle((0,0),1,1, color=color_map[label]) for label in labels]
    custom_labels = labels  # Use provided labels directly
    
    # Display legend with custom labels and handles
    plt.legend(custom_handles, custom_labels, loc='upper center', fontsize=12)
    
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.show()

# Example usage:
labels = ['PyTorch', 'TVM', 'Autotune_conv2d', 'Autotune_all']
plot_timing_results(timing_results_list, labels, model='vit', dataset='ImageNet')
